In [1]:
import sys
sys.path.append('..')

from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
from enum import Enum

from src.database.session import SessionLocal
from src.utils import *
from src.crud.utils import get_performance
from src.models import *
from src import crud


db = SessionLocal()
end_date_plus_one = date(year=2023, month=8, day=1)
end_date = end_date_plus_one - timedelta(days=1)
start_date = end_date_plus_one - relativedelta(months=12)
shop_id = 25963515

2023-08-12 07:42:59.825 | DEBUG    | src.database.session:<module>:17 - base: prod.cw6tznt1p9hq.us-east-1.rds.amazonaws.com
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = crud.ga_daily_performance.get_performance(
    db=db,
    shop_id=shop_id,
    start_date=start_date,
    end_date=end_date,
)

In [5]:
df['type']

0            SHOPPING
1     PERFORMANCE_MAX
2     PERFORMANCE_MAX
3     PERFORMANCE_MAX
4     PERFORMANCE_MAX
5     PERFORMANCE_MAX
6     PERFORMANCE_MAX
7     PERFORMANCE_MAX
8     PERFORMANCE_MAX
9     PERFORMANCE_MAX
10    PERFORMANCE_MAX
11    PERFORMANCE_MAX
Name: type, dtype: object

In [4]:
df[['spend', 'spend_USD']]

,spend,spend_USD
0,10186.000000,70.497210
1,97156.026987,672.415950
2,304702.395828,2108.842418
3,8184.203371,56.642795
4,303620.708097,2101.356067
5,90538.698419,626.617481
6,97204.270671,672.749844
7,303669.313971,2101.692468
8,60864.364441,421.241694
9,302854.993672,2096.056565


In [3]:
df['spend_USD']

0      1875.75
1      5870.98
2      6698.21
3      9765.78
4       231.19
        ...   
283      53.01
284      55.74
285      55.80
286      54.28
287     122.12
Name: spend_USD, Length: 288, dtype: float64

In [ ]:
ccr_subquery = (
            db.query(CurrencyExchangeRate.code, CurrencyExchangeRate.rate_from_usd)
            .order_by(CurrencyExchangeRate.code, CurrencyExchangeRate.date.desc())
            .distinct(CurrencyExchangeRate.code)
        ).subquery()

In [14]:
conversion_rates_subquery = (
    db.query(CurrencyExchangeRate.code, CurrencyExchangeRate.rate_from_usd)
    .order_by(CurrencyExchangeRate.code, CurrencyExchangeRate.date.desc())
    .distinct(CurrencyExchangeRate.code)
).subquery()

query = (
    db.query(FacebookAdAccount.facebook_id, FacebookAdAccount.currency, conversion_rates_subquery.c.rate_from_usd)
    .join(conversion_rates_subquery, conversion_rates_subquery.c.code == FacebookAdAccount.currency)
    .filter(FacebookAdAccount.shop_id == shop_id)
)

conversion_dict = {row.facebook_id: row.rate_from_usd for row in query.all()}

In [20]:
money_columns = ['spend', 'purch_value']
for column in money_columns:
    df[f'{column}_USD'] = df.apply(lambda df_: df_[column] * conversion_dict[df_['account_id']], axis=1)

In [17]:
conversion_dict = {row.facebook_id: row.rate_from_usd for row in ccr_query.all()}

In [7]:
df_old = ping_creative(db=db, shop_id=16038, start_date=start_date, end_date=end_date)

In [8]:
df_new = ping_creative2(db=db, shop_id=16038, start_date=start_date, end_date=end_date)